## Import library

In [16]:
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 

from kan import *
import warnings
import sys
sys.path.append('../utils')
from treasury_base import *

warnings.filterwarnings("ignore")

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Retrieve data

In [12]:
WINDOW_LIST = [5, 10]

def train_mse():
    predictions = model(dataset['train_input'])  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']) # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5
    
df_ma = ma_data_retrieval(WINDOW_LIST)
df_ma.head()

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,...,5 Yr_MA5,5 Yr_MA10,7 Yr_MA5,7 Yr_MA10,10 Yr_MA5,10 Yr_MA10,20 Yr_MA5,20 Yr_MA10,30 Yr_MA5,30 Yr_MA10
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-16,2.41,2.40,2.43,2.49,2.57,2.55,2.53,2.54,2.62,2.73,...,2.542,2.517,2.616,2.587,2.724,2.694,2.910,2.870,3.054,3.013
2019-01-17,2.41,2.41,2.42,2.50,2.57,2.56,2.55,2.58,2.66,2.75,...,2.536,2.522,2.612,2.593,2.722,2.701,2.914,2.879,3.062,3.023
2019-01-18,2.40,2.40,2.41,2.50,2.60,2.62,2.60,2.62,2.70,2.79,...,2.540,2.543,2.618,2.615,2.724,2.720,2.916,2.897,3.064,3.038
2019-01-22,2.38,2.40,2.43,2.51,2.59,2.58,2.55,2.57,2.65,2.74,...,2.560,2.556,2.638,2.629,2.740,2.732,2.926,2.909,3.074,3.049
2019-01-23,2.37,2.38,2.41,2.51,2.59,2.58,2.57,2.59,2.66,2.76,...,2.568,2.560,2.648,2.634,2.746,2.736,2.926,2.914,3.074,3.056


## KAN model training

In [17]:
TEST_SIZE = 1
LENGTH = len(df_ma)
TARGETS = df_ma.columns[:12]

# Store results for each fold
fold_results = {'train_mse': [], 'test_mse': [], 'naive_mse': []}

for cnt in range(0, 20, 5):
    print()
    print('WINDOW SLIDING: ', cnt)

    df_window = df_ma[(LENGTH-cnt-120):(LENGTH-cnt)]
    # Prepare data
    X, y = df_window.drop(columns=TARGETS), df_window[TARGETS]

    # scaler = StandardScaler()
    # X = pd.DataFrame(scaler.fit_transform(X))

    n_inputs = X.shape[1]
    n_outputs = y.shape[1]

    X_train, X_test = X[:-TEST_SIZE], X[-TEST_SIZE:]
    y_train, y_test = y[:-TEST_SIZE], y[-TEST_SIZE:]

    dataset = dict()
    dtype = torch.get_default_dtype()
    dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
    dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
    dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
    dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

    # Initialize the model
    model = KAN(width=[n_inputs, 32, n_outputs], grid=3, k=2, seed=42, device=device)

    # Train the model and compute metrics
    results = model.fit(dataset, opt="Adam", lamb=0.001, lr=0.001, steps=1500, metrics=(train_mse, test_mse))
    df_naive = pd.DataFrame([y_train.iloc[-1]] * TEST_SIZE, columns=y_train.columns)
        
    # Store the metrics
    train_error = results['train_mse'][-1]
    test_error = results['test_mse'][-1]
    naive_error = mean_squared_error(df_naive, y_test, squared=False)

    fold_results['train_mse'].append(train_error)
    fold_results['test_mse'].append(test_error)
    fold_results['naive_mse'].append(naive_error)

    # Calculate average metrics across all windows
    print(f'Fold Train MSE: {train_error}')
    print(f'Fold Test MSE: {test_error}')
    print(f'Naive Test MSE: {naive_error}')

avg_train_mse = np.mean(fold_results['train_mse'])
avg_test_mse = np.mean(fold_results['test_mse'])
avg_naive_mse = np.mean(fold_results['naive_mse'])

print()
print("Sliding Window Cross-Validation Results")
print(f"Average Train MSE: {avg_train_mse}")
print(f"Average Test MSE: {avg_test_mse}")
print(f"Average Naive MSE: {avg_naive_mse}")


WINDOW SLIDING:  0
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 4.09e+00 | test_loss: 3.86e+00 | reg: 9.25e+01 | :   0%|     | 0/1500 [00:00<?, ?it/s]

| train_loss: 5.18e-02 | test_loss: 7.96e-02 | reg: 1.87e+01 | : 100%|█| 1500/1500 [00:34<00:00, 43.


saving model version 0.1
Fold Train MSE: 0.05150229584130992
Fold Test MSE: 0.07955366019740276
Naive Test MSE: 0.031666666666666655

WINDOW SLIDING:  5
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 5.58e-02 | test_loss: 1.22e-01 | reg: 1.83e+01 | : 100%|█| 1500/1500 [00:34<00:00, 43.


saving model version 0.1
Fold Train MSE: 0.055799943404819356
Fold Test MSE: 0.1219883361632906
Naive Test MSE: 0.04666666666666671

WINDOW SLIDING:  10
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 5.58e-02 | test_loss: 2.67e-02 | reg: 1.78e+01 | : 100%|█| 1500/1500 [00:34<00:00, 44.


saving model version 0.1
Fold Train MSE: 0.05574805819963644
Fold Test MSE: 0.026698057746989533
Naive Test MSE: 0.024166666666666597

WINDOW SLIDING:  15
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 5.36e-02 | test_loss: 4.05e-02 | reg: 1.94e+01 | : 100%|█| 1500/1500 [00:34<00:00, 43.


saving model version 0.1
Fold Train MSE: 0.05382796506586082
Fold Test MSE: 0.04049469683766699
Naive Test MSE: 0.029166666666666785

Sliding Window Cross-Validation Results
Average Train MSE: 0.054219565627906634
Average Test MSE: 0.06718368773633747
Average Naive MSE: 0.03291666666666669
